# Рекомендация тарифов

**Цели и описание проекта:**

Есть данные оператора мобильной связи о клиентах, использующих новые тарифы мобильной связи "Смарт" и "Ультра" в течение месяца. Задача оператора - перевести клиентов, которые еще пользуются архивными тарифами на один из новых тарифов. Нам нужно построить модель, которая выберет подходящий тариф. Будем решать задачу классификации, обучать модель с учителем.

План работы:

- разобьем данные на выборки (обучающую, валидационную и тестовую)
- исследуем разные модели (дерева решений, случайного леса, логистической регрессии)
- проверим модели на тестовой выборке
- проверим модель на адекватность

В результате проекта мы должны построить модель с максимально большим значением accuracy, не ниже 0.75.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.dummy import DummyClassifier

pd.set_option('display.max_columns', None)

In [2]:
try:
    df = pd.read_csv('/datasets/users_behavior.csv')
except:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/users_behavior.csv')

df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [3]:
df['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
# поменяем тип данных в столбцах с количеством звонков и сообщений
df = df.astype({'calls': np.int64, 'messages': np.int64})

In [6]:
df.dtypes

calls         int64
minutes     float64
messages      int64
mb_used     float64
is_ultra      int64
dtype: object

**Вывод:**
- Исходный датасет содержит данные о 3214 клиентах. 
- Клиентов, использующих тариф ультра почти в два раза меньше, чем тех, кто использует тариф смарт. 
- Один столбец содержит категориальные значения - вид тарифа, остальные количественные значения - количество звонков, сообщений, минут и мегабайт. 
- Пропуски не обнаружены. 
-Мы изменили тип данных на целочисленный в столбцах с количеством звонков и сообщений. 

## Разделение данных на выборки

In [7]:
# Разделим данные на обучающую, тестовую и валидационную выборки в пропорции 60:20:20

# Сначала выделим обучающую и создадим промежуточную выборки
df_train, df_test1 = train_test_split(df, test_size=0.4,
                                      stratify = df['is_ultra'],
                                      random_state=12345)

#Теперь разделим промежуточную выборку на валидационную и тестовую
df_test, df_valid = train_test_split(df_test1, test_size=0.5,
                                     stratify = df_test1['is_ultra'],
                                     random_state=12345)

In [8]:
df_train.shape

(1928, 5)

In [9]:
df_test.shape

(643, 5)

In [10]:
df_valid.shape

(643, 5)

In [11]:
# Создадим переменные с признаками для обучающей выборки
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

In [12]:
# Создадим переменные с признаками для валидационной выборки
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

In [13]:
# Создадим переменные с признаками для тестовой выборки
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

**Вывод:**
На этом шаге мы разбили датасет на три выборки: обучающую, валидационную и тестовую, в пропорции 60/20/20. Случайное число **random_state** будем использовать **12345**. Также мы подготовили переменные с признаками для дальнейшей работы с моделями.


## Исследование моделей

### Дерево решений

In [14]:
# Создадим и исследуем модель решающее дерево
# Выберем лучший показатель Accuracy среди разных зачений глубины дерева
best_model_dtc = None
best_result_dtc = 0

for depth in range(1, 10):
    model_dtc = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_dtc.fit(features_train, target_train)
    predictions_dtc = model_dtc.predict(features_valid)
    result_dtc = accuracy_score(predictions_dtc, target_valid)
    if result_dtc > best_result_dtc:
        best_model_dtc = model_dtc
        best_result_dtc = result_dtc
        
print("Лучшая модель DTC:", best_model_dtc)
print("Accuracy лучшей модели DTC:", best_result_dtc)

Лучшая модель DTC: DecisionTreeClassifier(max_depth=5, random_state=12345)
Accuracy лучшей модели DTC: 0.8118195956454122


**Вывод:**

  Лучший показатель *Accuracy* среди моделей решающего дерева -  **0.8118195956454122**. 
  Глубина лучшей модели решающего дерева *max_depth* равна **5**

### Случайный лес

In [15]:
# Создадим и исследуем модель случайный лес
# Выберем лучший Accuracy среди разного количества деревьев от 1 до 10
best_model_rfc = None
best_result_rfc = 0

for est in range(1, 11):
    model_rfc = RandomForestClassifier(random_state=12345, n_estimators=est)
    model_rfc.fit(features_train, target_train)
    predictions_rfc = model_rfc.predict(features_valid)
    result_rfc = accuracy_score(predictions_rfc, target_valid)
    if result_rfc > best_result_rfc:
        best_model_rfc = model_rfc
        best_result_rfc = result_rfc
        
print("Лучшая модель RFC:", best_model_rfc)
print("Accuracy лучшей модели RFC:", best_result_rfc)   

Лучшая модель RFC: RandomForestClassifier(n_estimators=9, random_state=12345)
Accuracy лучшей модели RFC: 0.7947122861586314


**Вывод:**

  Лучший показатель *Accuracy* среди моделей слуайного леса -  **0.7947122861586314**. 
  Количество деревьев модели случайного леса *n_estimators* равно **9**

### Логистическая регрессия

In [16]:
# Создадим и исследуем модель логистической регрессии
# Зададим гиперпараметры solver='lbfgs' и max_iter=1000
model_lr = LogisticRegression(random_state=12345, solver='lbfgs',
                              max_iter=1000) 
model_lr.fit(features_train, target_train) 
accuracy_lr = model_lr.score(features_valid, target_valid)

print("Accuracy модели LR:", accuracy_lr)

Accuracy модели LR: 0.7465007776049767


**Вывод:**
Показатель *Accuracy* модели логистической регрессии равен **0.7465007776049767**

### Вывод

По результатам исследования моделей решающего дерева, случайного леса и логистической регрессии, наилучший показатель *Accuracy* **0.8118195956454122** выявлен у модели решающего дерева с параметром глубины *max_depth* равным **5**

## Проверка модели на тестовой выборке

In [17]:
# Проверим модель решающего дерева с наилучшим Accuracy на тестовой выборке
model = DecisionTreeClassifier(random_state=12345, max_depth=7) 
model.fit(features_train, target_train)
accuracy_test = model.score(features_test, target_test)

print("Accuracy модели DTC на тестовой выборке:", accuracy_test)

Accuracy модели DTC на тестовой выборке: 0.7869362363919129


**Вывод:**
Модель решающего дерева, показавшая лучший результат *Accuracy* на валидационной выборке, при проверке на тестовой выборке выдала *Accuracy* **0.7869362363919129**. Это значение соответствует условию данного проекта, оно не ниже 0,75 

## Проверка модели на адекватность

In [18]:
# Для проверки адекватности построим константную модель
# с параметром, который предсказывает наиболее часто встречающуюся метку в обучающем наборе.
model_dc = DummyClassifier(strategy="most_frequent", random_state=12345)
model_dc.fit(features_train, target_train)
accuracy_dc = model_dc.score(features_valid, target_valid)

print("Accuracy константной модели:", accuracy_dc)

Accuracy константной модели: 0.6936236391912908


**Вывод:**
Accuracy константной модели ниже, чем у модели решающего дерева. Значит модель решающего дерева адекватна.

## Общий вывод

- В ходе проекта были исследованы модели решающего дерева, случайного леса и логистической регрессии. Наилучший показатель *Accuracy* **0.8118195956454122** выявлен у модели решающего дерева с параметром глубины *max_depth* равным **5**
- Модель решающего дерева, показавшая лучший результат *Accuracy* на валидационной выборке, при проверке на тестовой выборке выдала *Accuracy* **0.7869362363919129**. Это значение соответствует условию данного проекта, оно не ниже 0,75
- Проверяя модель на адекватность, мы использовали константную модель. Accuracy константной модели оказалась ниже, чем у модели решающего дерева. Значит модель решающего дерева адекватна.